In [72]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import BSplines
from scipy.linalg import solveh_banded
from scipy.sparse.linalg import splu
from scipy.sparse import diags
import gpflow
import matplotlib.pyplot as plt
from gpflow.inducing_variables import InducingVariables
from gpflow.base import TensorLike
from gpflow.utilities import to_default_float
from gpflow import covariances as cov
from gpflow import kullback_leiblers as kl
from gpflow.ci_utils import ci_niter
import time


In [73]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


In [74]:
class SplineFeatures1D(InducingVariables):
    def __init__(self, a, b, M):
        """
        'a' and 'b' define the range over which the data is defined.
        'M' specifies the number of cubic B-spline basis functions to use.
        """

        # [a,b] defining the interval of the Spline representation
        self.a = gpflow.Parameter(a, dtype=gpflow.default_float())
        self.b = gpflow.Parameter(b, dtype=gpflow.default_float())
        self.M = M

        # integer array defining the knot sequence
        #self.knots = gpflow.Parameter(np.linspace(self.a, self.b, M + 1 + 3)[:, None], name='knots')
        self.knots = np.linspace(self.a, self.b, M + 1 + 3)

    def num_inducing(self):
        """
        Number of inducing variables (defines the dimensionality of q(u))
        """
        return self.M

In [75]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

In [76]:
@cov.Kuu.register(SplineFeatures1D, gpflow.kernels.Matern12)
def Kuu_matern12_splinefeatures1d(inducing_variable, kernel, jitter=1):
    knots = inducing_variable.knots
    
    # J = tf.zeros([inducing_variable.num_inducing(),inducing_variable.num_inducing()], dtype=tf.float64)
    # diagonal = np.repeat(jitter, inducing_variable.num_inducing())
    # J = tf.linalg.set_diag(J, diagonal)

    Kuu = BSplines.Matern12_Kuu(knots, kernel.lengthscales, kernel.variance)
    return Kuu

@cov.Kuf.register(SplineFeatures1D, gpflow.kernels.Matern12, TensorLike)
def Kuf_matern12_splinefeatures1d(inducing_variable, kernel, X):
    X = tf.squeeze(X, axis=1)
    knots = inducing_variable.knots
    Kuf = BSplines.Matern12_Kuf(knots, X.numpy())
    return Kuf

In [82]:
num_inducing = 1000
X = np.linspace(-1.9, 1.9, 30).reshape(-1,1)
Kuf = Kuf_matern12_splinefeatures1d(SplineFeatures1D(-2,2,num_inducing), gpflow.kernels.Matern12(), X)
Kuu = Kuu_matern12_splinefeatures1d(SplineFeatures1D(-2,2,num_inducing), gpflow.kernels.Matern12())

#is_pos_def(Kuu.numpy())

Kuu_ab = tf.linalg.diag_part(Kuu, k=(-3,0)).numpy()
KuuInv_Kuf = tf.convert_to_tensor(solveh_banded(Kuu_ab, Kuf.numpy(), lower=True))

In [81]:
num_inducing = 2000
X = np.linspace(-1.9, 1.9, 30).reshape(-1,1)
Kuf = Kuf_matern12_splinefeatures1d(SplineFeatures1D(-2,2,num_inducing), gpflow.kernels.Matern12(), X)
Kuu = Kuu_matern12_splinefeatures1d(SplineFeatures1D(-2,2,num_inducing), gpflow.kernels.Matern12())

## We now need Kuu solve AX=b

ab = tf.linalg.diag_part(Kuu, k=(-3,0)).numpy()
b = tf.convert_to_tensor(np.random.normal(0,1,num_inducing))
x = tf.convert_to_tensor(solveh_banded(ab, b.numpy().reshape(-1,1), lower=True))

print(x.shape)

## Log determinant of Kuu

t0 = time.time()
"""Tensorflow implmetation (1)"""
Kuu_LO = tf.linalg.LinearOperatorFullMatrix(Kuu)
K_ld = Kuu_LO.log_abs_determinant()
print(K_ld)

t1 = time.time()
"""Tensorflow implmetation (2)"""
K_ld = tf.linalg.slogdet(Kuu)
print(K_ld)

t2 = time.time()

"""Scipy Sparse implementation"""
elements = Kuu[0:7,3].numpy()
Kuu_sparse = diags(elements, [-3, -2, -1, 0, 1, 2, 3], shape=(Kuu.shape), format='csc')

t3 = time.time()
lu = splu(Kuu_sparse)
diagL = lu.L.diagonal().astype(np.complex128)
diagU = lu.U.diagonal().astype(np.complex128)
logdet = np.log(diagL).sum() + np.log(diagU).sum()
print(tf.convert_to_tensor(logdet))

t4 = time.time()
"""numpy implementation"""
print(np.linalg.slogdet(Kuu.numpy()))

t5 = time.time()




LinAlgError: 7th leading minor not positive definite

In [17]:
num_inducing = 2000
X = np.linspace(-1.9, 1.9, 30).reshape(-1,1)
Kuf = Kuf_matern12_splinefeatures1d(SplineFeatures1D(-2,2,num_inducing), gpflow.kernels.Matern12(), X)
Kuu = Kuu_matern12_splinefeatures1d(SplineFeatures1D(-2,2,num_inducing), gpflow.kernels.Matern12())

Kuu_ab = tf.linalg.diag_part(Kuu, k=(-3,0)).numpy()
KuuInv_Kuf = tf.convert_to_tensor(solveh_banded(Kuu_ab, Kuf.numpy(), lower=True))

LinAlgError: 7th leading minor not positive definite

In [23]:
@kl.prior_kl.register(SplineFeatures1D, gpflow.kernels.Kernel, TensorLike, TensorLike)
def prior_kl_vff(inducing_variable, kernel, q_mu, q_sqrt, whiten=False):
    if whiten:
        raise NotImplementedError
    K = cov.Kuu(inducing_variable, kernel)
    return gauss_kl_vff(q_mu, q_sqrt, K)


def gauss_kl_vff(q_mu, q_sqrt, K):
    """
    Compute the KL divergence from

          q(x) = N(q_mu, q_sqrt^2)
    to
          p(x) = N(0, K)

    q_mu is a vector [N, 1] that contains the mean.
    q_sqrt is a matrix that is the lower triangular square-root matrix of the covariance of q.

    K is a positive definite matrix: the covariance of p.
    NOTE: K is a LinearOperator that provides efficient methjods
        for solve(), log_abs_determinant(), and trace()
    """
    # KL(N₀ || N₁) = ½ [tr(Σ₁⁻¹ Σ₀) + (μ₁ - μ₀)ᵀ Σ₁⁻¹ (μ₁ - μ₀) - k + ln(det(Σ₁)/det(Σ₀))]
    # N₀ = q; μ₀ = q_mu, Σ₀ = q_sqrt q_sqrtᵀ
    # N₁ = p; μ₁ = 0, Σ₁ = K
    # KL(q || p) =
    #     ½ [tr(K⁻¹ q_sqrt q_sqrtᵀA + q_muᵀ K⁻¹ q_mu - k + logdet(K) - logdet(q_sqrt q_sqrtᵀ)]
    # k = number of dimensions, if q_sqrt is m x m this is m²


### THIS HAS NOT YET BEEN OPTIMISED FOR SPECIFIC MATRIX STRUCTURE ###

    """Kinv_q_mu = K.solve(q_mu)"""
    ab = tf.linalg.diag_part(K, k=(-3,0)).numpy()
    Kinv_q_mu = tf.convert_to_tensor(solveh_banded(ab, q_mu.numpy().reshape(-1,1), lower=True))
    """"""

    mahalanobis_term = tf.squeeze(tf.matmul(q_mu, Kinv_q_mu, transpose_a=True))

    # GPflow: q_sqrt is num_latent_gps x N x N
    num_latent_gps = to_default_float(tf.shape(q_mu)[1])

    """K.log_abs_determinant()"""
    elements = Kuu[0:7,3].numpy()
    Kuu_sparse = diags(elements, [-3, -2, -1, 0, 1, 2, 3], shape=(Kuu.shape), format='csc')
    lu = splu(Kuu_sparse)
    diagL = lu.L.diagonal().astype(np.complex128)
    diagU = lu.U.diagonal().astype(np.complex128)
    k_log_det = np.log(diagL).sum() + np.log(diagU).sum()
    """"""

    logdet_prior = num_latent_gps * k_log_det

    product_of_dimensions__int = tf.reduce_prod(tf.shape(q_sqrt)[:-1])  # dimensions are integers
    constant_term = to_default_float(product_of_dimensions__int)

    Lq = tf.linalg.band_part(q_sqrt, -1, 0)  # force lower triangle
    logdet_q = tf.reduce_sum(tf.math.log(tf.square(tf.linalg.diag_part(Lq))))

    """K.solve(Lq)"""
    Kinv_Lq = tf.convert_to_tensor(solveh_banded(ab, Lq.numpy().reshape(-1,1), lower=True))
    """"""

    # S = tf.matmul(q_sqrt, q_sqrt, transpose_b=True)
    # trace_term = tf.trace(K.solve(S))
    trace_term = tf.squeeze(
        tf.reduce_sum(Lq * Kinv_Lq, axis=[-1, -2])
    )  # [O(N²) instead of O(N³)

    twoKL = trace_term + mahalanobis_term - constant_term + logdet_prior - logdet_q
    return 0.5 * twoKL

In [41]:
import gpflow.posteriors


class CGPPosterior(gpflow.posteriors.BasePosterior):
    def _conditional_fused(self, Xnew, full_cov, full_output_cov):
        """
        Xnew is a tensor with the points of the data or minibatch, shape N x D
        """
        if full_output_cov:
            raise NotImplementedError

        f = self._q_dist.q_mu
        q_sqrt = self._q_dist.q_sqrt

        # num_data = tf.shape(Xnew)[0]  # M
        num_func = tf.shape(f)[1]  # K

        Kuu = cov.Kuu(self.X_data, self.kernel)  # this is now a LinearOperator
        Kuf = cov.Kuf(self.X_data, self.kernel, Xnew)  # still a Tensor

        """KuuInv_Kuf = Kuu.solve(Kuf)"""
        Kuu_ab = tf.linalg.diag_part(Kuu, k=(-3,0)).numpy()
        KuuInv_Kuf = tf.convert_to_tensor(solveh_banded(Kuu_ab, Kuf.numpy(), lower=True))


        # compute the covariance due to the conditioning
        if full_cov:
            fvar = self.kernel(Xnew) - tf.matmul(Kuf, KuuInv_Kuf, transpose_a=True)
            shape = (num_func, 1, 1)
        else:
            KufT_KuuInv_Kuf_diag = tf.reduce_sum(Kuf * KuuInv_Kuf, axis=-2)
            fvar = self.kernel(Xnew, full_cov=False) - KufT_KuuInv_Kuf_diag
            shape = (num_func, 1)
        fvar = tf.expand_dims(fvar, 0) * tf.ones(
            shape, dtype=gpflow.default_float()
        )  # K x N x N or K x N

        if self.whiten:
            raise NotImplementedError

        A = KuuInv_Kuf

        # construct the conditional mean
        fmean = tf.matmul(A, f, transpose_a=True)

        if q_sqrt is not None:
            if q_sqrt.get_shape().ndims == 2:
                # LTA = A * tf.expand_dims(q_sqrt, 2)  # K x M x N
                # won't work  # make ticket for this?
                raise NotImplementedError
            elif q_sqrt.get_shape().ndims == 3:
                # L = tf.matrix_band_part(tf.transpose(q_sqrt, (2, 0, 1)), -1, 0)  # K x M x M

                # K x M x N
                # A_tiled = tf.expand_dims(A.get(), 0) * tf.ones((num_func, 1, 1), dtype=float_type)

                # LTA = tf.matmul(L, A_tiled, transpose_a=True)  # K x M x N
                # TODO the following won't work for K > 1
                assert q_sqrt.shape[0] == 1
                # LTA = (A.T @ DenseMatrix(q_sqrt[:,:,0])).T.get()[None, :, :]
                ATL = tf.matmul(A, q_sqrt, transpose_a=True)
            else:
                raise ValueError("Bad dimension for q_sqrt: %s" % str(q_sqrt.get_shape().ndims))
            if full_cov:
                # fvar = fvar + tf.matmul(LTA, LTA, transpose_a=True)  # K x N x N
                fvar = fvar + tf.matmul(ATL, ATL, transpose_b=True)  # K x N x N
            else:
                # fvar = fvar + tf.reduce_sum(tf.square(LTA), 1)  # K x N
                fvar = fvar + tf.reduce_sum(tf.square(ATL), 2)  # K x N
        fvar = tf.transpose(fvar)  # N x K or N x N x K

        return fmean, fvar

    # We can also provide a conditional that precomputes as much as possible,
    # to speed up predictions:

    def _precompute(self):
        Kuu = cov.Kuu(self.X_data, self.kernel)  # this is now a LinearOperator

        q_mu = self._q_dist.q_mu
        q_sqrt = self._q_dist.q_sqrt

        if self.whiten:
            raise NotImplementedError
        else:
            # alpha = Kuu⁻¹ q_mu
            alpha = Kuu.solve(q_mu)  # type: tf.Tensor

        if self.whiten:
            raise NotImplementedError
        else:
            # Qinv = Kuu⁻¹ - Kuu⁻¹ S Kuu⁻¹
            KuuInv_qsqrt = Kuu.solve(q_sqrt)
            KuuInv_covu_KuuInv = tf.matmul(KuuInv_qsqrt, KuuInv_qsqrt, transpose_b=True)

        Qinv = Kuu.inverse().to_dense() - KuuInv_covu_KuuInv

        return alpha, Qinv

    def _conditional_with_precompute(self, cache, Xnew, full_cov, full_output_cov):
        alpha, Qinv = cache

        if full_output_cov:
            raise NotImplementedError

        Kuf = cov.Kuf(self.X_data, self.kernel, Xnew)  # still a Tensor

        # construct the conditional mean
        fmean = tf.matmul(Kuf, alpha, transpose_a=True)

        num_func = tf.shape(alpha)[1]  # K
        Qinv_Kuf = tf.matmul(Qinv, Kuf)

        # compute the covariance due to the conditioning
        if full_cov:
            fvar = self.kernel(Xnew) - tf.matmul(Kuf, Qinv_Kuf, transpose_a=True)
        else:
            KufT_Qinv_Kuf_diag = tf.reduce_sum(Kuf * Qinv_Kuf, axis=-2)
            fvar = self.kernel(Xnew, full_cov=False) - KufT_Qinv_Kuf_diag
        fvar = tf.transpose(fvar)

        return fmean, fvar

In [42]:
@gpflow.posteriors.get_posterior_class.register(gpflow.kernels.Kernel, SplineFeatures1D)
def _get_posterior_vff(kernel, inducing_variable):
    return CGPPosterior

In [43]:
Mf = 2
M = 2 * Mf - 1
kernel = gpflow.kernels.Matern12()
inducing_variable = SplineFeatures1D(-0.5, 1.5, Mf)

Xnew = np.random.rand(7, 1)
f = np.random.randn(M, 1)
q_sqrt = tf.convert_to_tensor(np.tril(np.random.randn(1, M, M)))

conditional_f_mean, conditional_f_var = gpflow.conditionals.conditional(
    Xnew, inducing_variable, kernel, f, q_sqrt=q_sqrt, white=False, full_cov=True
)

posterior = CGPPosterior(kernel, inducing_variable, f, q_sqrt, whiten=False, precompute_cache=None)
posterior_f_mean, posterior_f_var = posterior.fused_predict_f(Xnew, full_cov=True)

np.testing.assert_array_equal(conditional_f_mean, posterior_f_mean)
np.testing.assert_array_equal(conditional_f_var, posterior_f_var)

InvalidArgumentError: Matrix size-incompatible: In[0]: [2,7], In[1]: [3,1] [Op:MatMul]